# ScholarGenie: Summarization & Analysis Demo

This notebook demonstrates the complete pipeline: finding papers, parsing PDFs, generating summaries, and creating presentations.

## Setup

In [ ]:
import sys
sys.path.append('..')

from backend.agents.paper_finder import PaperFinderAgent
from backend.agents.pdf_parser import PDFParserAgent
from backend.agents.summarizer import SummarizerAgent
from backend.agents.extractor import ExtractorAgent
from backend.agents.presenter import PresenterAgent
from backend.agents.evaluator import EvaluatorAgent

print("✅ All agents imported successfully")

## Step 1: Find a Paper

In [ ]:
finder = PaperFinderAgent()

# Get the famous "Attention Is All You Need" paper
arxiv_id = "1706.03762"
paper_data = finder.get_paper_by_arxiv_id(arxiv_id)

print(f"Found: {paper_data['title']}")
print(f"PDF URL: {paper_data['pdf_url']}")

## Step 2: Parse PDF

In [ ]:
parser = PDFParserAgent()

# Download and parse
print("📄 Downloading and parsing PDF...")
paper = parser.download_and_parse(
    pdf_url=paper_data['pdf_url'],
    paper_id=paper_data['paper_id']
)

print(f"\n✅ Parsed successfully!")
print(f"Title: {paper.title}")
print(f"Authors: {', '.join([a.name for a in paper.authors[:3]])}")
print(f"Sections: {len(paper.sections)}")
print(f"References: {len(paper.references)}")
print(f"Figures: {len(paper.figures)}")

## Step 3: View Content

In [ ]:
# View abstract
print("Abstract:")
print(paper.abstract)

In [ ]:
# View section titles
print("Sections:")
for i, section in enumerate(paper.sections, 1):
    print(f"{i}. {section.title}")

## Step 4: Generate Summary

In [ ]:
summarizer = SummarizerAgent()

print("📝 Generating multi-granularity summary...")
print("(This may take a few minutes depending on your hardware)\n")

summary = summarizer.summarize_paper(paper)

print("✅ Summary generated!")

In [ ]:
# View TL;DR
print("TL;DR:")
print(summary.tldr)
print()

# View short summary
print("Short Summary:")
print(summary.short_summary)
print()

# View full summary
print("Full Summary:")
print(summary.full_summary)

In [ ]:
# View key points
print("Key Points:")
for i, point in enumerate(summary.keypoints, 1):
    print(f"{i}. {point}")

## Step 5: Extract Structured Data

In [ ]:
extractor = ExtractorAgent()

print("🔍 Extracting structured data...")
extracted = extractor.extract(paper)

print("✅ Extraction complete!")

In [ ]:
# View extracted data
print("Methods:", extracted.methods[:5])
print("\nDatasets:", extracted.datasets[:5])
print("\nModels:", extracted.models[:5])
print("\nKey Findings:")
for finding in extracted.key_findings[:3]:
    print(f"- {finding}")

## Step 6: Evaluate Summary Quality

In [ ]:
evaluator = EvaluatorAgent()

print("📊 Evaluating summary quality...")
evaluation = evaluator.evaluate_summary(paper, summary)

print("\n✅ Evaluation complete!")

In [ ]:
# View ROUGE scores
if 'rouge' in evaluation['metrics']:
    print("ROUGE Scores:")
    for metric, scores in evaluation['metrics']['rouge'].items():
        print(f"{metric}: F1={scores['fmeasure']:.3f}, P={scores['precision']:.3f}, R={scores['recall']:.3f}")

In [ ]:
# View quality checks
print("\nQuality Checks:")
qc = evaluation['quality_checks']

print(f"TL;DR length: {qc['tldr_length']['word_count']} words - {'✅' if qc['tldr_length']['pass'] else '❌'}")
print(f"Full summary length: {qc['full_summary_length']['word_count']} words - {'✅' if qc['full_summary_length']['pass'] else '❌'}")
print(f"Keypoints count: {qc['keypoints_count']['count']} - {'✅' if qc['keypoints_count']['pass'] else '❌'}")
print(f"Has methods: {'✅' if qc['has_methods'] else '❌'}")
print(f"Has results: {'✅' if qc['has_results'] else '❌'}")

In [ ]:
# View warnings
if evaluation['warnings']:
    print("\nWarnings:")
    for warning in evaluation['warnings']:
        print(f"⚠️  {warning}")
else:
    print("\n✅ No warnings!")

## Step 7: Generate Presentation

In [ ]:
presenter = PresenterAgent()

print("🎨 Generating PowerPoint presentation...")

pptx_path = presenter.generate_pptx(
    paper=paper,
    summary=summary,
    extracted_data=extracted,
    output_path="./attention_presentation.pptx"
)

print(f"\n✅ Presentation saved to: {pptx_path}")

## Step 8: Generate Report

In [ ]:
print("📄 Generating Markdown report...")

report_path = presenter.generate_markdown_report(
    paper=paper,
    summary=summary,
    extracted_data=extracted,
    output_path="./attention_report.md"
)

print(f"\n✅ Report saved to: {report_path}")

In [ ]:
# Preview report
with open(report_path, 'r', encoding='utf-8') as f:
    report_content = f.read()

from IPython.display import Markdown
Markdown(report_content[:2000] + "\n\n... (truncated)")

## Visualize Summary Statistics

In [ ]:
import matplotlib.pyplot as plt

# Summary lengths
summary_lengths = {
    'TL;DR': len(summary.tldr.split()),
    'Short': len(summary.short_summary.split()),
    'Full': len(summary.full_summary.split()),
    'Keypoints': sum(len(kp.split()) for kp in summary.keypoints)
}

plt.figure(figsize=(10, 5))
plt.bar(summary_lengths.keys(), summary_lengths.values(), color='skyblue')
plt.xlabel('Summary Type')
plt.ylabel('Word Count')
plt.title('Summary Length Comparison')
plt.tight_layout()
plt.show()

## Conclusion

You've completed the full ScholarGenie pipeline! You can now:

1. ✅ Find papers from arXiv and Semantic Scholar
2. ✅ Parse PDFs with GROBID
3. ✅ Generate multi-level summaries
4. ✅ Extract structured data
5. ✅ Create presentations and reports
6. ✅ Evaluate summary quality

### Next Steps:

- Try with your own papers
- Experiment with different models in `config.yaml`
- Use the vector store for semantic search
- Batch process multiple papers
- Deploy with Docker for production use